# Simple Backup Script for NX-OS
Requirements:
 - Must have SSH key generated on switch, and public key copied to backup server
 - Enable NXAPI and NXAPI-Sandbox on N7k
 
Program Flow: 
 - Script runs as a cron job once daily on backup server.
 - Script connects to switch via NXAPI.
 - Script issues command: copy startup-config sftp://10.18.188.188/home/bkup/n7k/[name]-[date]
   - N7k copies startup-config to the backup server

On Switch: 
 - conf t
 - feature nxapi
 - feature nxapi-sandbox
 - username admin keypair generate rsa
 - show username admin keypair
 export keypair
 copy public key to bkup server
 add to authorized_keys
 
 test manual config copy

In [22]:
import requests
import json
import sys
from datetime import date
from requests.auth import HTTPBasicAuth

In [11]:
## Variables
today = str(date.today())

In [14]:
print(auth)

In [15]:
show_config_payload = {
  "ins_api": {
    "version": "1.2",
    "type": "cli_show",
    "chunk": "0",
    "sid": "1",
    "input": "show running-config",
    "output_format": "json"
  }
}

In [29]:
# This should be moved to a config file that can be read/parsed.
# Each tuple is a device hostname, ip address, and credentials.

devices = [
    ('N7K1-Labcore01','10.18.188.137','admin','password123'),
    ('N7K2-Labcore02','10.18.188.138','admin','correct horse battery staple'),
    ('N7K1-ACILab01','10.18.188.141','admin','password123'),
    ('N7K1-ACILab02','10.18.188.142','admin','correct horse battery staple'),       
    ('IPN1','10.18.188.143','admin','password123'),
    ('IPN2','10.18.188.144','admin','correct horse battery staple'),        
    ('MSITE1','10.18.188.145','admin','password123'),         
    ('MSITE2','10.18.188.146','admin','correct horse battery staple')
]

In [30]:
bkup_host = "10.18.188.188"
bkup_user = "bkup"
proto_schema = "sftp"
bkup_device = ""
bkup_source_file = "startup-config"
default_vrf = "vrf management"

In [31]:
for device in devices:
        hostname = device[0]
        ip = device[1]
        username = device[2]
        password = device[3]
        #print(hostname + " " + ip + " " + username + " " + password)
        bkup_path = "/home/bkup/n7k/" + hostname + "/"
        bkup_dest_file = bkup_source_file + "_" + hostname + "_" + today + "-TESTING "
        input_cmd = "copy " + bkup_source_file + " " + proto_schema + "://" + bkup_user + "@" + bkup_host + bkup_path + bkup_dest_file + default_vrf
        #fprint(input_cmd)
        bkup_payload = {
            "ins_api": {
            "version": "1.2",
            "type": "cli_conf",
            "chunk": "0",
            "sid": "1",
            "input": input_cmd,
            "output_format": "json"
            }
        }
        

        auth = HTTPBasicAuth(username, password)
        
        nx_url = "http://" + ip + "/ins"
        #r = requests.post(auth_url, json=auth_data, verify=False)
        print("Running backup command: ")
        print(input_cmd)
        print("at " + nx_url)
        r = requests.post(nx_url, auth=auth, json=bkup_payload)
        if r.status_code != 200:
            print(r.status_code)
            sys.exit()

Running backup command: 
copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/N7K1-Labcore01/startup-config_N7K1-Labcore01_2019-07-08-TESTING vrf management
at http://10.18.188.137/ins
Running backup command: 
copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/N7K2-Labcore02/startup-config_N7K2-Labcore02_2019-07-08-TESTING vrf management
at http://10.18.188.138/ins
Running backup command: 
copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/N7K1-ACILab01/startup-config_N7K1-ACILab01_2019-07-08-TESTING vrf management
at http://10.18.188.141/ins
Running backup command: 
copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/N7K1-ACILab02/startup-config_N7K1-ACILab02_2019-07-08-TESTING vrf management
at http://10.18.188.142/ins
Running backup command: 
copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/IPN1/startup-config_IPN1_2019-07-08-TESTING vrf management
at http://10.18.188.143/ins
Running backup command: 
copy startup-config sftp://bkup@10.18.1

In [19]:
%tb

SystemExit: 

200

In [15]:
r.headers

{'Server': 'nginx/1.4.5', 'Date': 'Mon, 08 Jul 2019 17:40:09 GMT', 'Content-Type': 'text/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'nxapi_auth=admin:156260820189115991'}

In [21]:
#r.json
xml = r.content

In [22]:
r_json = r.json()

In [23]:
r_json

{'ins_api': {'sid': 'eoc',
  'type': 'cli_conf',
  'version': '1.2',
  'outputs': {'output': {'code': '500',
    'msg': 'Backend processing error',
    'input': 'copy startup-config sftp://bkup@10.18.188.188/home/bkup/n7k/startup-config-2019-06-26'}}}}